In [2]:
import neo4j, langchain_neo4j
print("neo4j", neo4j.__version__)
print("langchain_neo4j", langchain_neo4j.__version__)

from langchain_neo4j import Neo4jGraph
graph = Neo4jGraph(url=NEO4J_URI, username=NEO4J_USERNAME, password=NEO4J_PASSWORD)
graph.query("RETURN 1 AS ok")


neo4j 5.28.3
langchain_neo4j 0.8.0


[{'ok': 1}]

In [3]:
moview_query="""
LOAD CSV WITH HEADERS FROM
'https://raw.githubusercontent.com/tomasonjo/blog-datasets/main/movies/movies_small.csv' as row

MERGE(m:Movie{id:row.movieId})
SET m.released = date(row.released),
    m.title = row.title,
    m.imdbRating = toFloat(row.imdbRating)
FOREACH (director in split(row.director, '|') | 
    MERGE (p:Person {name:trim(director)})
    MERGE (p)-[:DIRECTED]->(m))
FOREACH (actor in split(row.actors, '|') | 
    MERGE (p:Person {name:trim(actor)})
    MERGE (p)-[:ACTED_IN]->(m))
FOREACH (genre in split(row.genres, '|') | 
    MERGE (g:Genre {name:trim(genre)})
    MERGE (m)-[:IN_GENRE]->(g))


"""
graph.query(moview_query)

[]

In [4]:
import os
from dotenv import load_dotenv
load_dotenv()

groq_api_key=os.getenv("GROQ_API_KEY")

from langchain_groq import ChatGroq
llm=ChatGroq(groq_api_key=groq_api_key,model_name="llama-3.1-8b-instant")
llm

ChatGroq(profile={'max_input_tokens': 131072, 'max_output_tokens': 8192, 'image_inputs': False, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': False, 'tool_calling': True}, client=<groq.resources.chat.completions.Completions object at 0x123dbc4a0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x124111310>, model_name='llama-3.1-8b-instant', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [5]:
from langchain_neo4j.chains.graph_qa.cypher import GraphCypherQAChain

chain = GraphCypherQAChain.from_llm(
    graph=graph,
    llm=llm,
    verbose=True,
    validate_cypher=True,
    allow_dangerous_requests=True,
)


In [6]:
chain.graph_schema

'Node properties:\nUser {name: STRING, userId: INTEGER}\nPost {postId: INTEGER, content: STRING, timestamp: DATE_TIME}\nMovie {id: STRING, released: DATE, title: STRING, imdbRating: FLOAT}\nPerson {name: STRING}\nGenre {name: STRING}\nRelationship properties:\n\nThe relationships:\n(:User)-[:POSTED]->(:Post)\n(:User)-[:FRIEND]->(:User)\n(:User)-[:LIKES]->(:User)\n(:Movie)-[:IN_GENRE]->(:Genre)\n(:Person)-[:DIRECTED]->(:Movie)\n(:Person)-[:ACTED_IN]->(:Movie)'

In [24]:
from langchain_core.prompts import FewShotPromptTemplate, PromptTemplate

examples = [
    {
        "question": "How many artists are there?",
        "query": "MATCH (a:Person)-[:ACTED_IN]->(:Movie) RETURN count(DISTINCT a) AS actors",
    },
    {
        "question": "Which actors played in the movie Casino?",
        "query": "MATCH (m:Movie {{title: 'Casino'}})<-[:ACTED_IN]-(a:Person) RETURN a.name AS actor",
    },
    {
        "question": "How many movies has Tom Hanks acted in?",
        "query": "MATCH (a:Person {{name: 'Tom Hanks'}})-[:ACTED_IN]->(m:Movie) RETURN count(m) AS movies",
    },
    {
        "question": "List all the genres of the movie Schindler's List",
        "query": "MATCH (m:Movie {{title: 'Schindler\\'s List'}})-[:IN_GENRE]->(g:Genre) RETURN g.name AS genre",
    },
    {
        "question": "Which actors have worked in movies from both the comedy and action genres?",
        "query": "MATCH (a:Person)-[:ACTED_IN]->(:Movie)-[:IN_GENRE]->(g1:Genre), (a)-[:ACTED_IN]->(:Movie)-[:IN_GENRE]->(g2:Genre) WHERE g1.name = 'Comedy' AND g2.name = 'Action' RETURN DISTINCT a.name AS actor",
    },
    {
        "question": "Which actors acted in more than one movie?",
        "query": "MATCH (a:Person)-[:ACTED_IN]->(m:Movie) WITH a, count(DISTINCT m) AS movies WHERE movies > 1 RETURN a.name AS actor, movies AS count"
    }   

]

example_prompt = PromptTemplate.from_template(
    "User input: {question}\nCypher query: {query}"
)

prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix=(
        "You are a Neo4j expert. Given an input question, create a syntactically accurate Cypher query.\n"
        "Use only the schema below.\n\nSchema:\n{schema}\n"
    ),
    suffix="User input: {question}\nCypher query:",
    input_variables=["question", "schema"],
)

print(prompt.format(question="How many artists are there?", schema="foo"))


You are a Neo4j expert. Given an input question, create a syntactically accurate Cypher query.
Use only the schema below.

Schema:
foo


User input: How many artists are there?
Cypher query: MATCH (a:Person)-[:ACTED_IN]->(:Movie) RETURN count(DISTINCT a) AS actors

User input: Which actors played in the movie Casino?
Cypher query: MATCH (m:Movie {title: 'Casino'})<-[:ACTED_IN]-(a:Person) RETURN a.name AS actor

User input: How many movies has Tom Hanks acted in?
Cypher query: MATCH (a:Person {name: 'Tom Hanks'})-[:ACTED_IN]->(m:Movie) RETURN count(m) AS movies

User input: List all the genres of the movie Schindler's List
Cypher query: MATCH (m:Movie {title: 'Schindler\'s List'})-[:IN_GENRE]->(g:Genre) RETURN g.name AS genre

User input: Which actors have worked in movies from both the comedy and action genres?
Cypher query: MATCH (a:Person)-[:ACTED_IN]->(:Movie)-[:IN_GENRE]->(g1:Genre), (a)-[:ACTED_IN]->(:Movie)-[:IN_GENRE]->(g2:Genre) WHERE g1.name = 'Comedy' AND g2.name = 'Action' R

In [25]:
from langchain_core.prompts import FewShotPromptTemplate, PromptTemplate

example_prompt = PromptTemplate.from_template(
    "User input: {question}\nCypher query: {query}"
)

prompt = FewShotPromptTemplate(
    examples=examples[:5],
    example_prompt=example_prompt,
    prefix=(
        "Task: Generate a Cypher statement to query a Neo4j graph database.\n"
        "Instructions:\n"
        "- Use only the provided schema.\n"
        "- Do not include any explanations or extra text.\n"
        "- Do not wrap the query in quotes or backticks.\n"
        "- Output ONLY the Cypher statement.\n\n"
        "Schema:\n{schema}\n"
    ),
    suffix="User input: {question}\nCypher query:",
    input_variables=["schema", "question"],
)


In [26]:
from langchain_neo4j.chains.graph_qa.cypher import GraphCypherQAChain

graph.refresh_schema()

chain = GraphCypherQAChain.from_llm(
    graph=graph,
    llm=llm,
    validate_cypher=True,
    return_direct=True,
    allow_dangerous_requests=True,
    verbose=True,
    cypher_llm_kwargs={
        "prompt": prompt,                 # <-- your FewShotPromptTemplate
        "stop": ["\n\n", "```"],           # optional: reduces extra commentary
    },
)


In [27]:
graph.refresh_schema()
print(graph.schema)


Node properties:
User {name: STRING, userId: INTEGER}
Post {postId: INTEGER, content: STRING, timestamp: DATE_TIME}
Movie {id: STRING, released: DATE, title: STRING, imdbRating: FLOAT}
Person {name: STRING}
Genre {name: STRING}
Relationship properties:

The relationships:
(:User)-[:POSTED]->(:Post)
(:User)-[:FRIEND]->(:User)
(:User)-[:LIKES]->(:User)
(:Movie)-[:IN_GENRE]->(:Genre)
(:Person)-[:DIRECTED]->(:Movie)
(:Person)-[:ACTED_IN]->(:Movie)


In [19]:
chain.invoke({"query": "How many movies are there?"})
chain.invoke({"query": "How many actors are there?"})
chain.invoke({"query": "Which actors played in the movie Casino?"})




> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (m:Movie) RETURN count(m) AS movies

> Finished chain.


> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (a:Person)-[:ACTED_IN]->(:Movie) RETURN count(DISTINCT a) AS actors

> Finished chain.


> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (m:Movie {title: 'Casino'})<-[:ACTED_IN]-(a:Person) RETURN a.name AS actor

> Finished chain.


{'query': 'Which actors played in the movie Casino?',
 'result': [{'actor': 'Robert De Niro'},
  {'actor': 'Joe Pesci'},
  {'actor': 'Sharon Stone'},
  {'actor': 'James Woods'}]}

In [20]:
chain.invoke("Which actors played in the movie Casino?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (m:Movie {title: 'Casino'})<-[:ACTED_IN]-(a:Person) RETURN a.name AS actor

> Finished chain.


{'query': 'Which actors played in the movie Casino?',
 'result': [{'actor': 'Robert De Niro'},
  {'actor': 'Joe Pesci'},
  {'actor': 'Sharon Stone'},
  {'actor': 'James Woods'}]}

In [22]:
chain.invoke({"query": "actors who acted in multiple movies"})



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (a:Person)-[:ACTED_IN]->(m:Movie)-[:IN_GENRE]->(g:Genre) WITH a, COUNT(DISTINCT m) AS movies RETURN a.name AS actor, movies AS count

> Finished chain.


{'query': 'actors who acted in multiple movies',
 'result': [{'actor': 'Jim Varney', 'count': 1},
  {'actor': 'Tim Allen', 'count': 2},
  {'actor': 'Tom Hanks', 'count': 2},
  {'actor': 'Don Rickles', 'count': 1},
  {'actor': 'Robin Williams', 'count': 2},
  {'actor': 'Bradley Pierce', 'count': 1},
  {'actor': 'Kirsten Dunst', 'count': 1},
  {'actor': 'Jonathan Hyde', 'count': 1},
  {'actor': 'Jonathan Taylor Thomas', 'count': 2},
  {'actor': 'Brad Renfro', 'count': 2}]}

In [28]:
chain.invoke({"query": "actors who acted in multiple movies"})




> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (a:Person)-[:ACTED_IN]->(m:Movie) RETURN a.name AS actor, count(DISTINCT m) AS movies ORDER BY movies DESC

> Finished chain.


{'query': 'actors who acted in multiple movies',
 'result': [{'actor': 'Gene Hackman', 'movies': 4},
  {'actor': 'John Travolta', 'movies': 4},
  {'actor': 'Julianne Moore', 'movies': 4},
  {'actor': 'Robert Downey Jr.', 'movies': 4},
  {'actor': 'Christian Slater', 'movies': 4},
  {'actor': 'Samuel L. Jackson', 'movies': 4},
  {'actor': 'Gary Oldman', 'movies': 4},
  {'actor': 'Angela Bassett', 'movies': 3},
  {'actor': 'Robert De Niro', 'movies': 3},
  {'actor': 'Julia Ormond', 'movies': 3}]}

In [29]:
res = chain.invoke({"query": "Which actors acted in more than one movie?"})




> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (a:Person)-[:ACTED_IN]->(m:Movie) RETURN count(DISTINCT a) AS actors

> Finished chain.
